In [1]:
import numpy as np

In [2]:
identifier = 'LSTMGAN-BS56LR0.01GR10SeqLen30Sepsis'

In [3]:
real_data_Sepsis = np.load('./data/'+identifier+'real_data.npy')

In [4]:
fake_data_Sepsis = np.load('./data/'+identifier+'fake_data.npy')

In [5]:
real_data_Sepsis.shape

(2627, 30, 5)

In [6]:
fake_data_Sepsis.shape

(2184, 30, 5)

In [7]:
identifier = 'LSTMGAN-BS56LR0.01GR10SeqLen30-Non-Sepsis'

In [8]:
real_data_NonSepsis = np.load('./data/'+identifier+'real_data.npy')

In [9]:
fake_data_NonSepsis = np.load('./data/'+identifier+'fake_data.npy')

In [10]:
real_data_NonSepsis.shape

(1314, 30, 5)

In [11]:
fake_data_NonSepsis.shape

(1064, 30, 5)

## Train on Real

In [12]:
## 0 - Non-Sepsis
## 1 - Sepsis

In [13]:
real_data_X = np.vstack((real_data_NonSepsis,real_data_Sepsis))
real_data_X.shape

(3941, 30, 5)

In [14]:
real_data_Y = np.concatenate([np.zeros(real_data_NonSepsis.shape[0]),np.ones(real_data_Sepsis.shape[0])])
real_data_Y.shape

(3941,)

In [15]:
seq_len = real_data_Sepsis.shape[1]
feature_len = real_data_Sepsis.shape[2]

In [16]:
print(seq_len)
print(feature_len)

30
5


## Model

In [17]:
import tensorflow as tf

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [19]:
model = Sequential()
model.add(LSTM(100, input_shape=(seq_len, feature_len)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               42400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 42,501
Trainable params: 42,501
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_val, y_train, y_val = train_test_split(real_data_X, real_data_Y, test_size=0.2, random_state=42)

In [23]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(3152, 30, 5)
(789, 30, 5)
(3152,)
(789,)


In [24]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

Train on 3152 samples, validate on 789 samples
Epoch 1/10
3152/3152 [==============================] - 37s 12ms/step - loss: 0.6185 - acc: 0.6555 - val_loss: 0.5905 - val_acc: 0.6743
Epoch 2/10
3152/3152 [==============================] - 26s 8ms/step - loss: 0.5672 - acc: 0.7030 - val_loss: 0.5571 - val_acc: 0.7186
Epoch 3/10
3152/3152 [==============================] - 25s 8ms/step - loss: 0.5372 - acc: 0.7287 - val_loss: 0.5357 - val_acc: 0.7275
Epoch 4/10
3152/3152 [==============================] - 26s 8ms/step - loss: 0.5211 - acc: 0.7383 - val_loss: 0.5284 - val_acc: 0.7288
Epoch 5/10
3152/3152 [==============================] - 26s 8ms/step - loss: 0.5048 - acc: 0.7506 - val_loss: 0.5325 - val_acc: 0.7351
Epoch 6/10
3152/3152 [==============================] - 27s 9ms/step - loss: 0.4793 - acc: 0.7700 - val_loss: 0.5207 - val_acc: 0.7364
Epoch 7/10
3152/3152 [==============================] - 25s 8ms/step - loss: 0.4538 - acc: 0.7903 - val_loss: 0.4672 - val_acc: 0.7668
Epoch 8

## Test on Synthetic

In [25]:
fake_data_X = np.vstack((fake_data_NonSepsis,fake_data_Sepsis))
fake_data_X.shape

(3248, 30, 5)

In [26]:
fake_data_Y = np.concatenate([np.zeros(fake_data_NonSepsis.shape[0]),np.ones(fake_data_Sepsis.shape[0])])
fake_data_Y.shape

(3248,)

In [27]:
scores = model.evaluate(real_data_X, real_data_Y, verbose=1)

3941/3941 [==============================] - 9s 2ms/step


In [28]:
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 84.70%
